In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [3]:
pd.set_option('display.max_columns', None)

# Import data and data understanding

In [4]:
df = pd.read_csv('../../data/processed/combined data/combined.csv')

## Drop unnecessary column, dupicates and Nan

In [5]:
# Drop these columns as they are not useful
df = df.drop(columns=[
    'legId', 
    'searchDate',
    'isBasicEconomy',
    'segmentsEquipmentDescription',
    'segmentsAirlineCode',
    'segmentsAirlineName',
    'isRefundable',
    'isRefundable',
    'isNonStop',
    'segmentsDepartureTimeEpochSeconds',
    'segmentsArrivalTimeEpochSeconds',
    'segmentsDistance',
    
])

In [6]:
df = df.dropna()
df = df.drop_duplicates()

In [7]:
df

,flightDate,startingAirport,destinationAirport,travelDuration,totalFare,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsDurationInSeconds,segmentsCabinCode
0,2022-05-13,OAK,ATL,PT6H38M,317.60,2281.00,2022-05-13T11:42:00.000-07:00||2022-05-13T14:0...,2022-05-13T13:00:00.000-07:00||2022-05-13T21:2...,LAX||ATL,OAK||LAX,4680||15600,coach||coach
5,2022-05-13,OAK,ATL,PT9H3M,487.60,2175.00,2022-05-13T17:18:00.000-07:00||2022-05-13T23:4...,2022-05-13T20:15:00.000-06:00||2022-05-14T05:2...,SLC||ATL,OAK||SLC,7020||13260,coach||coach
8,2022-05-13,OAK,ATL,PT12H29M,526.11,2393.00,2022-05-13T07:16:00.000-07:00||2022-05-13T11:2...,2022-05-13T08:40:00.000-07:00||2022-05-13T14:4...,LAX||DEN||ATL,OAK||LAX||DEN,5040||8160||10620,coach||coach||coach
9,2022-05-13,OAK,ATL,PT12H29M,531.71,2393.00,2022-05-13T07:16:00.000-07:00||2022-05-13T12:5...,2022-05-13T08:40:00.000-07:00||2022-05-13T16:1...,LAX||DEN||ATL,OAK||LAX||DEN,5040||8340||10620,coach||coach||coach
10,2022-05-13,OAK,ATL,PT7H52M,534.01,2150.00,2022-05-13T18:58:00.000-07:00||2022-05-14T00:5...,2022-05-13T22:31:00.000-06:00||2022-05-14T05:5...,DEN||ATL,OAK||DEN,9180||10620,coach||coach
...,...,...,...,...,...,...,...,...,...,...,...,...
13519956,2022-05-20,MIA,SFO,PT15H51M,480.20,3670.00,2022-05-20T20:48:00.000-04:00||2022-05-21T06:2...,2022-05-20T23:59:00.000-04:00||2022-05-21T09:3...,EWR||SFO,MIA||EWR,11460||22200,coach||coach
13519959,2022-05-20,MIA,SFO,PT12H33M,492.20,2685.00,2022-05-20T06:45:00.000-04:00||2022-05-20T15:1...,2022-05-20T10:01:00.000-06:00||2022-05-20T16:1...,SLC||SFO,MIA||SLC,18960||7380,coach||coach
13519960,2022-05-20,MIA,SFO,PT10H1M,507.10,2623.00,2022-05-20T15:25:00.000-04:00||2022-05-20T18:0...,2022-05-20T17:14:00.000-05:00||2022-05-20T19:0...,IAH||PHX||SFO,MIA||IAH||PHX,10140||10800||7800,coach||coach||coach
13519974,2022-05-20,MIA,SFO,PT13H54M,557.20,2731.00,2022-05-20T08:00:00.000-04:00||2022-05-20T16:4...,2022-05-20T09:52:00.000-04:00||2022-05-20T18:5...,ATL||SFO,MIA||ATL,6720||18540,coach||coach


# Travel time and date

## Departure date 

In [8]:
df_dep_date = df[['segmentsDepartureTimeRaw']]

In [9]:
df_dep_date['segmentsDepartureTimeRaw'] = df_dep_date['segmentsDepartureTimeRaw'].apply(lambda x: x.split('||')[0])

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/2334269756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dep_date['segmentsDepartureTimeRaw'] = df_dep_date['segmentsDepartureTimeRaw'].apply(lambda x: x.split('||')[0])


In [10]:
df_dep_date['segmentsDepartureTimeRaw'] = pd.to_datetime(df_dep_date['segmentsDepartureTimeRaw'], utc=True)

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/3367788186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dep_date['segmentsDepartureTimeRaw'] = pd.to_datetime(df_dep_date['segmentsDepartureTimeRaw'], utc=True)


In [11]:
df_dep_date['dep_day'] = df_dep_date['segmentsDepartureTimeRaw'].dt.day
df_dep_date['dep_month'] = df_dep_date['segmentsDepartureTimeRaw'].dt.month
df_dep_date['dep_hour'] = df_dep_date['segmentsDepartureTimeRaw'].dt.hour
df_dep_date['dep_minute'] = df_dep_date['segmentsDepartureTimeRaw'].dt.minute

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/2103855625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dep_date['dep_day'] = df_dep_date['segmentsDepartureTimeRaw'].dt.day
/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/2103855625.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dep_date['dep_month'] = df_dep_date['segmentsDepartureTimeRaw'].dt.month
/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/2103855625.py:3: SettingWithCopyWarning: 
A

In [12]:
df_dep_date = df_dep_date.drop(columns=['segmentsDepartureTimeRaw'])

## Arrival date 

In [13]:
df_ar_date = df[['segmentsArrivalTimeRaw']]

In [14]:
df_ar_date['segmentsArrivalTimeRaw'] = df_ar_date['segmentsArrivalTimeRaw'].apply(lambda x: x.split('||')[-1])

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/4135951762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ar_date['segmentsArrivalTimeRaw'] = df_ar_date['segmentsArrivalTimeRaw'].apply(lambda x: x.split('||')[-1])


In [15]:
df_ar_date['segmentsArrivalTimeRaw'] = pd.to_datetime(df_ar_date['segmentsArrivalTimeRaw'], utc=True)

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/2069714180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ar_date['segmentsArrivalTimeRaw'] = pd.to_datetime(df_ar_date['segmentsArrivalTimeRaw'], utc=True)


In [16]:
df_ar_date['ar_day'] = df_ar_date['segmentsArrivalTimeRaw'].dt.day
df_ar_date['ar_month'] = df_ar_date['segmentsArrivalTimeRaw'].dt.month
df_ar_date['ar_hour'] = df_ar_date['segmentsArrivalTimeRaw'].dt.hour
df_ar_date['ar_minute'] = df_ar_date['segmentsArrivalTimeRaw'].dt.minute

df_ar_date = df_ar_date.drop(columns=['segmentsArrivalTimeRaw'])

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/3368766206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ar_date['ar_day'] = df_ar_date['segmentsArrivalTimeRaw'].dt.day
/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/3368766206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ar_date['ar_month'] = df_ar_date['segmentsArrivalTimeRaw'].dt.month
/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/3368766206.py:3: SettingWithCopyWarning: 
A value is 

# Travel and layover time

In [17]:
df_time = df[['travelDuration', 'segmentsDurationInSeconds']]

In [18]:
# Add the travel time together
df_time['totalTravelTimeInSeconds'] = df_time['segmentsDurationInSeconds'].apply(
    lambda x: sum(int(seg) for seg in x.split('||'))
)

# Delete the segmentsDurationInSeconds column
df_time = df_time.drop(['segmentsDurationInSeconds'], axis=1)

# Function to convert seconds to (days, hours, minutes)
def convert_seconds(seconds):
    days = seconds // 86400
    hours = (seconds % 86400) // 3600
    minutes = (seconds % 3600) // 60
    return f"{days}d {hours}h {minutes}m"

# Convert total travel time from seconds to (days, hours, minutes)
df_time['totalTravelTime'] = df_time['totalTravelTimeInSeconds'].apply(convert_seconds)

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/609739572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['totalTravelTimeInSeconds'] = df_time['segmentsDurationInSeconds'].apply(


In [19]:
# Convert total travel time from seconds to (days, hours, minutes)
df_time['totalTravelTime'] = df_time['totalTravelTimeInSeconds'].apply(convert_seconds)

# Delete the segmentsDurationInSeconds column
df_time = df_time.drop(['totalTravelTimeInSeconds'], axis=1)

In [20]:
# Convert travelDuration to timedelta
df_time['travelDuration'] = pd.to_timedelta(df_time['travelDuration'])

# Define the formatting function
def format_duration(td):
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes = remainder // 60
    return f"{days}d {hours}h {minutes}m"

# Apply the formatting function to the travelDuration column
df_time['travelDuration'] = df_time['travelDuration'].apply(format_duration)

In [21]:
# Convert travelDuration and totalTravelTime back to timedelta
df_time['travelDuration'] = pd.to_timedelta(df_time['travelDuration'].str.replace('d ', ' days '))
df_time['totalTravelTime'] = pd.to_timedelta(df_time['totalTravelTime'].str.replace('d ', ' days '))

# Calculate totalLayoverTime
df_time['totalLayoverTime'] = df_time['travelDuration'] - df_time['totalTravelTime']

df_time['totalLayoverTime'] = df_time['totalLayoverTime'].apply(format_duration)

In [22]:
df_time['totalLayoverTime'] = pd.to_timedelta(df_time['totalLayoverTime'].str.replace('d ', ' days '))

In [23]:
# Define the formatting function without seconds
def format_duration_no_seconds(td):
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes = remainder // 60
    return f"{days}d {hours}h {minutes}m"

# Apply the formatting function to each relevant column
df_time['travelDuration'] = df_time['travelDuration'].apply(format_duration_no_seconds)
df_time['totalTravelTime'] = df_time['totalTravelTime'].apply(format_duration_no_seconds)
df_time['totalLayoverTime'] = df_time['totalLayoverTime'].apply(format_duration_no_seconds)

## seperate layover time 

In [24]:
# Separate totalLayoverTime into days, hours, and minutes
df_time[['days', 'hours', 'minutes']] = df_time['totalLayoverTime'].str.extract(r'(\d+)d\s(\d+)h\s(\d+)m')

# Convert the columns to integers for further analysis if needed
df_time['days'] = df_time['days'].astype(int)
df_time['hours'] = df_time['hours'].astype(int)
df_time['minutes'] = df_time['minutes'].astype(int)

# Display the updated DataFrame
df_time.head()

,travelDuration,totalTravelTime,totalLayoverTime,days,hours,minutes
0,0d 6h 38m,0d 5h 38m,0d 1h 0m,0,1,0
5,0d 9h 3m,0d 5h 38m,0d 3h 25m,0,3,25
8,0d 12h 29m,0d 6h 37m,0d 5h 52m,0,5,52
9,0d 12h 29m,0d 6h 40m,0d 5h 49m,0,5,49
10,0d 7h 52m,0d 5h 30m,0d 2h 22m,0,2,22


In [25]:
df_time = df_time.drop(['days', 'totalTravelTime'], axis=1)
df_time = df_time.rename(columns={"hours": "layover_h", "minutes": "layover_m"})

In [26]:
df_layoverTime = df_time[['layover_h', 'layover_m']]

## seperate travel duration

In [27]:
# Separate totalLayoverTime into days, hours, and minutes
df_time[['days', 'hours', 'minutes']] = df_time['travelDuration'].str.extract(r'(\d+)d\s(\d+)h\s(\d+)m')

# Convert the columns to integers for further analysis if needed
df_time['duration_days'] = df_time['days'].astype(int)
df_time['duration_hours'] = df_time['hours'].astype(int)
df_time['duration_minutes'] = df_time['minutes'].astype(int)

In [28]:
df_duration = df_time[['duration_days', 'duration_hours', 'duration_minutes']]

# Layover

In [29]:
df_airport = df[['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode']]

In [30]:
df_airport['segmentsArrivalAirportCode'] = df_airport['segmentsArrivalAirportCode'].apply(lambda x: x.split('||')[-1])

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/407158195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_airport['segmentsArrivalAirportCode'] = df_airport['segmentsArrivalAirportCode'].apply(lambda x: x.split('||')[-1])


In [31]:
df_airport['route'] = df_airport['segmentsDepartureAirportCode'] + '||' + df_airport['segmentsArrivalAirportCode']
df_airport

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/3392405328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_airport['route'] = df_airport['segmentsDepartureAirportCode'] + '||' + df_airport['segmentsArrivalAirportCode']


,segmentsDepartureAirportCode,segmentsArrivalAirportCode,route
0,OAK||LAX,ATL,OAK||LAX||ATL
5,OAK||SLC,ATL,OAK||SLC||ATL
8,OAK||LAX||DEN,ATL,OAK||LAX||DEN||ATL
9,OAK||LAX||DEN,ATL,OAK||LAX||DEN||ATL
10,OAK||DEN,ATL,OAK||DEN||ATL
...,...,...,...
13519956,MIA||EWR,SFO,MIA||EWR||SFO
13519959,MIA||SLC,SFO,MIA||SLC||SFO
13519960,MIA||IAH||PHX,SFO,MIA||IAH||PHX||SFO
13519974,MIA||ATL,SFO,MIA||ATL||SFO


In [32]:
df_layover = df_airport[['route']]

In [33]:
# Calculate total layovers by counting '||' and subtracting 1 to get the number of stops
df_layover['totalLayover'] = df_layover['route'].apply(lambda x: x.count('||') - 1)

/var/folders/0h/hhtvsf517t90vww44wtg0nrw0000gn/T/ipykernel_48842/2569209673.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layover['totalLayover'] = df_layover['route'].apply(lambda x: x.count('||') - 1)


In [34]:
df_layover

,route,totalLayover
0,OAK||LAX||ATL,1
5,OAK||SLC||ATL,1
8,OAK||LAX||DEN||ATL,2
9,OAK||LAX||DEN||ATL,2
10,OAK||DEN||ATL,1
...,...,...
13519956,MIA||EWR||SFO,1
13519959,MIA||SLC||SFO,1
13519960,MIA||IAH||PHX||SFO,2
13519974,MIA||ATL||SFO,1


# depature and arrival dummy 

In [35]:
df_airport_dummy = df[['startingAirport', 'destinationAirport']]

In [36]:
# Create dummy variables with specific prefixes for startingAirport and destinationAirport
df_airport_dummy = pd.get_dummies(df_airport_dummy, 
                                  columns=['startingAirport', 'destinationAirport'], 
                                  prefix={'startingAirport': 'start_', 'destinationAirport': 'dest_'})

# Display the updated DataFrame
df_airport_dummy.head()

,start__ATL,start__BOS,start__CLT,start__DEN,start__DFW,start__DTW,start__EWR,start__IAD,start__JFK,start__LAX,start__LGA,start__MIA,start__OAK,start__ORD,start__PHL,start__SFO,dest__ATL,dest__BOS,dest__CLT,dest__DEN,dest__DFW,dest__DTW,dest__EWR,dest__IAD,dest__JFK,dest__LAX,dest__LGA,dest__MIA,dest__OAK,dest__ORD,dest__PHL,dest__SFO
0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [37]:
df_airport_dummy = df_airport_dummy.astype(int)

# Cabin encoding

In [38]:
df_cabin = df[['segmentsCabinCode']]

In [39]:
# Define cabin types
cabin_types = ['coach', 'business', 'first', 'premium coach']

# Function to count each cabin type
def count_cabin_types(cabin_code, cabin_types):
    counts = {cabin: 0 for cabin in cabin_types}
    segments = cabin_code.split('||')
    for segment in segments:
        if segment in counts:
            counts[segment] += 1
    return counts

In [40]:
# Apply function to create new columns
cabin_counts = df_cabin['segmentsCabinCode'].apply(lambda x: pd.Series(count_cabin_types(x, cabin_types)))
df_cabin = pd.concat([df_cabin, cabin_counts], axis=1)

In [41]:
df_cabin = df_cabin.drop(['segmentsCabinCode'], axis=1)

In [42]:
df_cabin

,coach,business,first,premium coach
0,2,0,0,0
5,2,0,0,0
8,3,0,0,0
9,3,0,0,0
10,2,0,0,0
...,...,...,...,...
13519956,2,0,0,0
13519959,2,0,0,0
13519960,3,0,0,0
13519974,2,0,0,0


# combined all different dataframe

In [43]:
s0 = df
s1 = df_dep_date
s2 = df_ar_date
s3 = df_layoverTime
s4 = df_duration
s5 = df_layover
s6 = df_airport_dummy
s7 = df_cabin


df_clean = pd.concat([s0, s1, s2, s3, s4, s5, s6, s7], axis=1)

In [44]:
df_clean = df_clean.drop(['flightDate',
                          'route',
                          'startingAirport', 
                          'destinationAirport', 
                          'travelDuration',
                          'segmentsDepartureTimeRaw', 
                          'segmentsArrivalTimeRaw',
                          'segmentsArrivalAirportCode',
                          'segmentsDepartureAirportCode',
                          'segmentsDurationInSeconds',
                          'segmentsCabinCode'], axis=1)

In [46]:
df_clean_test = df_clean.drop_duplicates()

In [47]:
df_clean_test

,totalFare,totalTravelDistance,dep_day,dep_month,dep_hour,dep_minute,ar_day,ar_month,ar_hour,ar_minute,layover_h,layover_m,duration_days,duration_hours,duration_minutes,route,totalLayover,start__ATL,start__BOS,start__CLT,start__DEN,start__DFW,start__DTW,start__EWR,start__IAD,start__JFK,start__LAX,start__LGA,start__MIA,start__OAK,start__ORD,start__PHL,start__SFO,dest__ATL,dest__BOS,dest__CLT,dest__DEN,dest__DFW,dest__DTW,dest__EWR,dest__IAD,dest__JFK,dest__LAX,dest__LGA,dest__MIA,dest__OAK,dest__ORD,dest__PHL,dest__SFO,coach,business,first,premium coach
0,317.60,2281.00,13,5,18,42,14,5,1,20,1,0,0,6,38,OAK||LAX||ATL,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
5,487.60,2175.00,14,5,0,18,14,5,9,21,3,25,0,9,3,OAK||SLC||ATL,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
8,526.11,2393.00,13,5,14,16,14,5,2,45,5,52,0,12,29,OAK||LAX||DEN||ATL,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0
9,531.71,2393.00,13,5,14,16,14,5,2,45,5,49,0,12,29,OAK||LAX||DEN||ATL,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0
10,534.01,2150.00,14,5,1,58,14,5,9,50,2,22,0,7,52,OAK||DEN||ATL,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13519956,480.20,3670.00,21,5,0,48,21,5,16,39,6,30,0,15,51,MIA||EWR||SFO,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0
13519959,492.20,2685.00,20,5,10,45,20,5,23,18,5,14,0,12,33,MIA||SLC||SFO,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0
13519960,507.10,2623.00,20,5,19,25,21,5,5,26,2,2,0,10,1,MIA||IAH||PHX||SFO,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0
13519974,557.20,2731.00,20,5,12,0,21,5,1,54,6,53,0,13,54,MIA||ATL||SFO,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0


# Model 

In [48]:
X = df_clean_test.drop(['totalFare'], axis=1)
y = df_clean_test[['totalFare']]

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [50]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [64]:
# Initialize the KNN regressor with chosen hyperparameters
knn_regressor = KNeighborsRegressor(n_neighbors=5, weights='distance', algorithm='auto')

# Train the model
knn_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 13196.197368800036
Mean Absolute Error: 66.98482920300582
R^2 Score: 0.7188270368049731


In [65]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest Regressor with chosen hyperparameters
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

/Users/leoli/Library/Caches/pypoetry/virtualenvs/at3-X3Atzmkc-py3.11/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Squared Error: 8629.387398575576
Mean Absolute Error: 59.40121507325385
R^2 Score: 0.8161326056586593
